In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import random
import pandas as pd
import PIL
from tqdm import tqdm

In [2]:
def set_seed(no):
    torch.manual_seed(no)
    random.seed(no)
    np.random.seed(no)
    os.environ['PYTHONHASHSEED'] = str()
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_seed(100)

In [3]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [4]:
# Hyper parameters
num_epochs = 20
batchsize = 128
lr = 0.0001

EPOCHS = 20
BATCH_SIZE = 90
LEARNING_RATE = 0.0001
TRAIN_DATA_PATH = "./Images"

In [5]:
train_data = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=data_transforms['train'])


In [6]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, 
                                           num_workers=8)

In [7]:
dinov2_vits14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')

Using cache found in /home/sagarnildass/.cache/torch/hub/facebookresearch_dinov2_main
xFormers not available
xFormers not available


In [8]:
class DinoVisionTransformerClassifier(nn.Module):
    def __init__(self):
        super(DinoVisionTransformerClassifier, self).__init__()
        self.transformer = dinov2_vits14
        self.classifier = nn.Sequential(
            nn.Linear(384, 256),
            nn.ReLU(),
            nn.Linear(256, 5)
        )
    
    def forward(self, x):
        x = self.transformer(x)
        x = self.transformer.norm(x)
        x = self.classifier(x)
        return x

     

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
import torch.optim as optim

model = DinoVisionTransformerClassifier()


criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
     

In [11]:
model = model.to(device)

In [12]:
def accuracy_fn(y_true, y_pred):
    """Calculates accuracy between truth labels and predictions.

    Args:
        y_true (torch.Tensor): Truth labels for predictions.
        y_pred (torch.Tensor): Predictions to be compared to predictions.

    Returns:
        [torch.float]: Accuracy value between y_true and y_pred, e.g. 78.45
    """
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

In [14]:
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    train_acc = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs.to(device))
        loss = criterion(outputs, labels.to(device))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        train_acc += accuracy_fn(y_true=labels.to(device),
                                 y_pred=outputs.argmax(dim=1)) 
        if i % 50 == 49:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 50:.3f}')
            running_loss = 0.0
    train_acc /= len(train_loader)
    print(f"Train accuracy: {train_acc:.2f}%")

print('Finished Training')
     

[1,    50] loss: 0.622
[1,   100] loss: 0.652
[1,   150] loss: 0.616
[1,   200] loss: 0.637
[1,   250] loss: 0.621
Train accuracy: 75.70%
[2,    50] loss: 0.624
[2,   100] loss: 0.585
[2,   150] loss: 0.603
[2,   200] loss: 0.583
[2,   250] loss: 0.588
Train accuracy: 77.47%
[3,    50] loss: 0.604
[3,   100] loss: 0.573
[3,   150] loss: 0.567
[3,   200] loss: 0.558
[3,   250] loss: 0.568
Train accuracy: 77.87%
[4,    50] loss: 0.552
[4,   100] loss: 0.536
[4,   150] loss: 0.556
[4,   200] loss: 0.574
[4,   250] loss: 0.530
Train accuracy: 79.27%
[5,    50] loss: 0.553
[5,   100] loss: 0.540
[5,   150] loss: 0.536
[5,   200] loss: 0.529
[5,   250] loss: 0.526
Train accuracy: 79.53%
Finished Training


In [15]:
from pathlib import Path

# Create models directory (if it doesn't already exist), see: https://docs.python.org/3/library/pathlib.html#pathlib.Path.mkdir
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, # create parent directories if needed
                 exist_ok=True # if models directory already exists, don't error
)

# Create model save path
MODEL_NAME = "x_ray_pretrain_dino_10_epochs.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# Save the model state dict
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model.state_dict(), # only saving the state_dict() only saves the learned parameters
           f=MODEL_SAVE_PATH)

Saving model to: models/x_ray_pretrain_dino_10_epochs.pth
